In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import datetime as td
import time

In [2]:
all_records = []
for page in range (1,6):
    url = f"https://api.coingecko.com/api/v3/coins/markets?vs_currency=inr&per_page=250&order=market_cap_desc&page=1&sparkline=False"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        all_records.extend(data)
#data

In [3]:
records = []
for i in data:
    records.append(dict(id = i['id'],
                       symbol = i['symbol'],
                        name = i['name'],
                        current_price = i['current_price'],
                        market_cap = i['market_cap'],
                        market_cap_rank = i['market_cap_rank'],
                        total_volume = i['total_volume'],
                        circulating_supply = i['circulating_supply'],
                        total_supply = i['total_supply'],
                        ath = i['ath'],
                        atl = i['atl'],
                        last_updated = i['last_updated']))

In [4]:
coins_df = pd.DataFrame(records)
coins_df['last_updated'] = pd.to_datetime(coins_df['last_updated']) # Convert the 'last_updated' column from string to datetime 
coins_df['date_only'] = coins_df['last_updated'].dt.date            # Extract just the date (YYYY-MM-DD)
coins_df

,id,symbol,name,current_price,market_cap,market_cap_rank,total_volume,circulating_supply,total_supply,ath,atl,last_updated,date_only
0,bitcoin,btc,Bitcoin,6451151.00,128895796936014,1,4.285350e+12,1.998616e+07,1.998616e+07,11187013.00,3993.420000,2026-02-08 14:33:25.088000+00:00,2026-02-08
1,ethereum,eth,Ethereum,192093.00,23171606628443,2,2.810596e+12,1.206926e+08,1.206926e+08,431946.00,28.130000,2026-02-08 14:33:25.561000+00:00,2026-02-08
2,tether,usdt,Tether,90.53,16813842963761,3,7.625520e+12,1.857160e+11,1.911830e+11,105.52,36.860000,2026-02-08 14:33:25.519000+00:00,2026-02-08
3,ripple,xrp,XRP,132.16,8049422158456,4,2.784230e+11,6.091732e+10,9.998572e+10,313.99,0.159343,2026-02-08 14:33:24.002000+00:00,2026-02-08
4,binancecoin,bnb,BNB,58341.00,7955196033699,5,1.106765e+11,1.363596e+08,1.363596e+08,121422.00,2.580000,2026-02-08 14:33:24.863000+00:00,2026-02-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,zero-gravity,0g,0G,49.39,10527801251,246,2.135465e+09,2.131997e+08,1.000000e+09,623.04,41.110000,2026-02-08 14:33:26.934000+00:00,2026-02-08
246,the-white-whale,whitewhale,The White Whale,10.36,10425459231,247,1.303441e+09,1.000000e+09,1.000000e+09,17.86,0.736516,2026-02-08 14:33:25.143000+00:00,2026-02-08
247,edu-coin,edu,Open Campus,12.49,10374343262,248,4.917363e+08,8.313750e+08,1.000000e+09,136.87,6.350000,2026-02-08 14:33:18.922000+00:00,2026-02-08
248,frax-usd,frxusd,Frax USD,90.55,10210676018,249,9.445563e+08,1.127600e+08,1.129016e+08,92.17,83.770000,2026-02-08 14:33:19.274000+00:00,2026-02-08


In [36]:
coin_list = ["bitcoin","ethereum","tether","binancecoin","ripple"]

all_data = []

for coin_id in coin_list:

    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart?vs_currency=inr&days=365&interval=daily"

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        df = pd.DataFrame(data['prices'], columns=['timestamp','price_inr'])

        df['date'] = pd.to_datetime(df['timestamp'], unit='ms').dt.date
        df['coin_id'] = coin_id

        df = df[['coin_id','date','price_inr']]
        all_data.append(df)

    else:
        print(f"Failed to fetch data for {coin_id}")

final_df = pd.concat(all_data)
final_df = final_df.drop_duplicates(subset=["coin_id","date"])         # Remove duplicates
final_df.reset_index(drop=True, inplace=True)

final_df

,coin_id,date,price_inr
0,bitcoin,2025-02-09,8.476775e+06
1,bitcoin,2025-02-10,8.468156e+06
2,bitcoin,2025-02-11,8.519902e+06
3,bitcoin,2025-02-12,8.311326e+06
4,bitcoin,2025-02-13,8.509968e+06
...,...,...,...
1820,ripple,2026-02-04,1.421941e+02
1821,ripple,2026-02-05,1.372055e+02
1822,ripple,2026-02-06,1.098331e+02
1823,ripple,2026-02-07,1.330935e+02


In [6]:

oil_df = pd.read_csv("https://raw.githubusercontent.com/datasets/oil-prices/main/data/wti-daily.csv")
#start_date = "2020-01-01"
#end_date = "2026-01-01"
oil_df["Date"] = pd.to_datetime(oil_df["Date"]) # Covert datetime - most CSV files it is actually stored as text (string), not a true datetime type.
oil_filtered = oil_df[(oil_df["Date"] >= "2020-01-01")&  # Filtering jan 2020 to jan 2026
                    (oil_df["Date"] <= "2026-01-01")]
oil_filtered

#oil_df.dtypes


,Date,Price
8569,2020-01-02,61.17
8570,2020-01-03,63.00
8571,2020-01-06,63.27
8572,2020-01-07,62.70
8573,2020-01-08,59.65
...,...,...
10064,2025-12-24,58.72
10065,2025-12-26,56.60
10066,2025-12-29,57.89
10067,2025-12-30,57.79


In [7]:
# Yfinance
tickers  = ["^GSPC", "^IXIC", "^NSEI"]
start_date = "2020-01-01"
end_date = "2026-01-12"

stocks_df = yf.download(tickers, start_date, end_date, group_by="tickers")
stocks_df.dropna(inplace=True)  # dropping Null values in all columns
stocks_df.head()

[*********************100%***********************]  3 of 3 completed


Ticker             ^NSEI                                                      \
Price               Open          High           Low         Close    Volume   
Date                                                                           
2020-01-02  12198.549805  12289.900391  12195.250000  12282.200195  407700.0   
2020-01-03  12261.099609  12265.599609  12191.349609  12226.650391  428800.0   
2020-01-06  12170.599609  12179.099609  11974.200195  11993.049805  396500.0   
2020-01-07  12079.099609  12152.150391  12005.349609  12052.950195  447800.0   
2020-01-08  11939.099609  12044.950195  11929.599609  12025.349609  446000.0   

Ticker            ^GSPC                                                       \
Price              Open         High          Low        Close        Volume   
Date                                                                           
2020-01-02  3244.669922  3258.139893  3235.530029  3257.850098  3.459930e+09   
2020-01-03  3226.360107  3246.149902  3222.340088  3234.850098  3.484700e+09   
2020-01-06  3217.550049  3246.840088  3214.639893  3246.280029  3.702460e+09   
2020-01-07  3241.860107  3244.909912  3232.429932  3237.179932  3.435910e+09   
2020-01-08  3238.590088  3267.070068  3236.669922  3253.050049  3.726840e+09   

Ticker            ^IXIC                                                       
Price              Open         High          Low        Close        Volume  
Date                                                                          
2020-01-02  9039.459961  9093.429688  9010.889648  9092.190430  2.862700e+09  
2020-01-03  8976.429688  9065.759766  8976.429688  9020.769531  2.586520e+09  
2020-01-06  8943.500000  9072.410156  8943.500000  9071.469727  2.810450e+09  
2020-01-07  9076.639648  9091.929688  9042.549805  9068.580078  2.381740e+09  
2020-01-08  9068.030273  9168.889648  9059.379883  9129.240234  2.472620e+09

In [8]:
#stocks_df["^GSPC"].reset_index()
GSPC_df = stocks_df["^GSPC"].reset_index()
GSPC_df["ticker"] = "^GSPC"

IXIC_df = stocks_df["^IXIC"].reset_index()
IXIC_df["ticker"] = "^IXIC"

NSEI_df = stocks_df["^NSEI"].reset_index()
NSEI_df["ticker"] = "^NSEI"

final_stocks = pd.concat([GSPC_df, IXIC_df, NSEI_df])

final_stocks

Price,Date,Open,High,Low,Close,Volume,ticker
0,2020-01-02,3244.669922,3258.139893,3235.530029,3257.850098,3.459930e+09,^GSPC
1,2020-01-03,3226.360107,3246.149902,3222.340088,3234.850098,3.484700e+09,^GSPC
2,2020-01-06,3217.550049,3246.840088,3214.639893,3246.280029,3.702460e+09,^GSPC
3,2020-01-07,3241.860107,3244.909912,3232.429932,3237.179932,3.435910e+09,^GSPC
4,2020-01-08,3238.590088,3267.070068,3236.669922,3253.050049,3.726840e+09,^GSPC
...,...,...,...,...,...,...,...
1439,2026-01-05,26333.699219,26373.199219,26210.050781,26250.300781,3.388000e+05,^NSEI
1440,2026-01-06,26189.699219,26273.949219,26124.750000,26178.699219,3.830000e+05,^NSEI
1441,2026-01-07,26143.099609,26187.150391,26067.900391,26140.750000,3.382000e+05,^NSEI
1442,2026-01-08,26106.500000,26133.199219,25858.449219,25876.849609,3.285000e+05,^NSEI


In [3]:
import pymysql

conn = pymysql.connect(
    host = "localhost",
    port = 3306,
    user = "root",
    password = "arun",
    database = "CrossMarket"
)
cursor = conn.cursor()

In [3]:
#cursor.execute("CREATE DATABASE CrossMarket")

In [4]:
cursor.execute(" use CrossMarket")

0

In [23]:
cursor.execute("""
CREATE TABLE Cryptocurrencies (
    id VARCHAR(100) PRIMARY KEY
        COMMENT 'Unique ID from CoinGecko (e.g., bitcoin)',
    symbol VARCHAR(10)
        COMMENT 'Short symbol (e.g., BTC, ETH)',
    name VARCHAR(100)
        COMMENT 'Full name of the cryptocurrency',
    current_price DECIMAL(18,6)
        COMMENT 'Current trading price (USD)',
    market_cap BIGINT
        COMMENT 'Market capitalization (USD)',
    market_cap_rank INT
        COMMENT 'Rank by market capitalization',
    total_volume BIGINT
        COMMENT 'Trading volume (24h, USD)',
    circulating_supply DECIMAL(30,6)
        COMMENT 'Coins in circulation',
    total_supply DECIMAL(30,6)
        COMMENT 'Total supply (if available)',
    ath DECIMAL(18,6)
        COMMENT 'All-time high price (USD)',
    atl DECIMAL(18,6)
        COMMENT 'All-time low price (USD)',
    date_only DATE
        COMMENT 'Last updated timestamp'
)
ENGINE=InnoDB;
""")


0

In [20]:
#query = "SELECT * FROM Cryptocurrencies"
#df_from_db = pd.read_sql(query, conn)
#print(df_from_db.head())

#cursor.execute("DROP DATABASE CrossMarket")
#conn.commit()

#print("Database deleted successfully")


Database deleted successfully


In [24]:
coins_df = coins_df.replace({np.nan: None})   #Replace NaN with None - Wherever data is missing, use None instead of NaN
coins_df = coins_df[[
    "id", "symbol", "name", "current_price", "market_cap", "market_cap_rank",
    "total_volume", "circulating_supply", "total_supply", "ath", "atl", "date_only"
]]

In [25]:
data = list(coins_df.itertuples(index=False, name=None))   #Convert DataFrame into rows -MySQL cursor cannot read a DataFrame directly

In [26]:
insert_query = """
INSERT INTO Cryptocurrencies (
    id, symbol, name, current_price, market_cap, market_cap_rank,
    total_volume, circulating_supply, total_supply, ath, atl, date_only
)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
ON DUPLICATE KEY UPDATE
    current_price = VALUES(current_price),
    market_cap = VALUES(market_cap),
    total_volume = VALUES(total_volume),
    circulating_supply = VALUES(circulating_supply),
    total_supply = VALUES(total_supply),
    date_only = VALUES(date_only)
"""
cursor.executemany(insert_query, data)
conn.commit()

In [27]:
cursor.execute("""
CREATE TABLE Crypto_prices (   
    coin_id VARCHAR(50)
        COMMENT 'Foreign key referencing Cryptocurrencies.id',        
    date DATE
        COMMENT 'Date of price record',        
    price_inr DECIMAL(18,6)
        COMMENT 'Price in INR',
    PRIMARY KEY (coin_id, date),

    CONSTRAINT fk_coin
    FOREIGN KEY (coin_id)
    REFERENCES Cryptocurrencies(id)
    ON DELETE CASCADE
)
ENGINE=InnoDB;        # storage engine to use to store table data - storage engine to use to store your table data
""")


0

In [37]:
final_df = (
    final_df[['coin_id','date','price_inr']]
    .where(pd.notnull(final_df), None)                         #Replace NaN with None - Wherever data is missing, use None instead of NaN
)

final_data = list(final_df.itertuples(index=False, name=None))  #Convert DataFrame into rows -MySQL cursor cannot read a DataFrame directly

    
insert_query = """
INSERT INTO Crypto_prices (coin_id, date, price_inr)
VALUES (%s, %s, %s)
ON DUPLICATE KEY UPDATE
price_inr = VALUES(price_inr)
"""
    
cursor.executemany(insert_query, final_data)
conn.commit()

In [38]:
cursor.execute(""" 
CREATE TABLE Oil_prices ( 
    Date DATE COMMENT 'Date of oil price', 
    Price DECIMAL(30,6)
)

""")

0

In [40]:
oil_filtered = oil_filtered[['Date', 'Price']]
oil_filtered = oil_filtered.where(pd.notnull(oil_filtered), None)
oil_data = list(oil_filtered.itertuples(index=False, name=None))

insert_query = """
INSERT INTO Oil_prices ( Date, Price) VALUES (%s,%s)
"""

cursor.executemany(insert_query, oil_data)
conn.commit()

In [43]:
cursor.execute("""
CREATE TABLE Stock_prices ( 
   Date DATE COMMENT 'Trading date', 
   Open DECIMAL(30, 6) COMMENT 'Opening price', 
   High DECIMAL(30, 6) COMMENT 'Highest price of the day',   
   Low DECIMAL(30, 6) COMMENT 'Lowest price of the day',
   Close DECIMAL(30, 6) COMMENT 'Closing price',
   Volume BIGINT COMMENT 'Trading volume',
   ticker VARCHAR(20) COMMENT 'Stock index symbol (e.g., ^GSPC, ^IXIC)'
)
""")

0

In [44]:
final_stocks = final_stocks[['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'ticker']]
final_stocks = final_stocks.where(pd.notnull(final_stocks), None)
stocks_data = list(final_stocks.itertuples(index=False, name=None))

In [46]:
insert_query = """
INSERT INTO Stock_prices ( Date, Open, High, Low, Close, Volume, ticker) VALUES (%s,%s,%s,%s,%s,%s,%s)
"""

cursor.executemany(insert_query, stocks_data)
conn.commit()

In [47]:
#Find the top 3 cryptocurrencies by market cap.

query1 = """
SELECT id, name, market_cap
FROM Cryptocurrencies
ORDER BY market_cap DESC
LIMIT 3;
"""

cursor.execute(query1)
print(cursor.fetchall())

(('bitcoin', 'Bitcoin', 128895796936014), ('ethereum', 'Ethereum', 23171606628443), ('tether', 'Tether', 16813842963761))


In [4]:
#List all coins where circulating supply exceeds 90% of total supply.

query2 = """
SELECT id, name, circulating_supply, total_supply
FROM Cryptocurrencies
WHERE circulating_supply >= 0.9 * total_supply;
"""

cursor.execute(query2)
print(cursor.fetchall())


(('1inch', '1INCH', Decimal('1407835446.291872'), Decimal('1499999999.997000')), ('a7a5', 'A7A5', Decimal('39188580304.190730'), Decimal('39188580304.190730')), ('aave', 'Aave', Decimal('15189922.731233'), Decimal('16000000.000000')), ('agora-dollar', 'AUSD', Decimal('216456853.000000'), Decimal('216456853.000000')), ('algorand', 'Algorand', Decimal('8869985383.001790'), Decimal('8870183454.857616')), ('ape-and-pepe', 'Ape and Pepe', Decimal('210000000000000.000000'), Decimal('210000000000000.000000')), ('apecoin', 'ApeCoin', Decimal('908664773.000000'), Decimal('1000000000.000000')), ('apenft', 'AINFT', Decimal('990105667256391.500000'), Decimal('990105667256391.500000')), ('apollo-diversified-credit-securitize-fund', 'Apollo Diversified Credit Securitize Fund', Decimal('124001.243716'), Decimal('124001.243716')), ('arweave', 'Arweave', Decimal('65454185.538151'), Decimal('65454185.538151')), ('astherus-usdf', 'Aster USDF', Decimal('163469630.002206'), Decimal('163599904.562206')), ('

In [ ]:
# Get coins that are within 10% of their all-time-high (ATH)

query3 = """
SELECT id, name, current_price, ath
FROM Cryptocurrencies
WHERE current_price >= 0.9 * ath;
"""

cursor.execute(query3)
print(cursor.fetchall())


In [50]:
#Find the average market cap rank of coins with volume above $1B.

query4 = """
SELECT AVG(market_cap_rank) AS avg_rank
FROM Cryptocurrencies
WHERE total_volume > 1000000000;
"""

cursor.execute(query4)
print(cursor.fetchall())


((Decimal('103.3087'),),)


In [51]:
# Get the most recently updated coin.

query5 = """
SELECT *
FROM Cryptocurrencies
ORDER BY date_only DESC
LIMIT 1;
"""

cursor.execute(query5)
print(cursor.fetchall())


(('1inch', '1inch', '1INCH', Decimal('9.060000'), 12763846634, 218, 1222766153, Decimal('1407835446.291872'), Decimal('1499999999.997000'), Decimal('648.390000'), Decimal('7.550000'), datetime.date(2026, 2, 8)),)


In [52]:
#Find the highest daily price of Bitcoin in the last 365 days.

query1 = """
SELECT MAX(price_inr) AS highest_price
FROM Crypto_prices
WHERE coin_id = 'bitcoin'
  AND date >= CURDATE() - INTERVAL 365 DAY;
"""

cursor.execute(query1)
print(cursor.fetchone())


(Decimal('11070472.246733'),)


In [53]:
#Calculate the average daily price of Ethereum in the past 1 year.

query2 = """
SELECT AVG(price_inr) AS avg_price
FROM Crypto_prices
WHERE coin_id = 'ethereum'
  AND date >= CURDATE() - INTERVAL 1 YEAR;
"""

cursor.execute(query2)
print(cursor.fetchone())


(Decimal('266191.0853786329'),)


In [ ]:
# Show the daily price trend of Bitcoin in March 2025.

query3 = """
SELECT date, price_inr
FROM Crypto_prices
WHERE coin_id = 'bitcoin'
  AND date BETWEEN '2025-03-01' AND '2025-03-31'
ORDER BY date;
"""

cursor.execute(query3)
print(cursor.fetchall())


In [55]:
# Find the coin with the highest average price over 1 year.

query4 = """
SELECT coin_id, AVG(price_inr) AS avg_price
FROM Crypto_prices
WHERE date >= CURDATE() - INTERVAL 1 YEAR
GROUP BY coin_id
ORDER BY avg_price DESC
LIMIT 1;
"""

cursor.execute(query4)
print(cursor.fetchone())


('bitcoin', Decimal('8784627.0446086932'))


In [8]:
#Get the % in Bitcoin’s price between Feb 2025 and Feb 2026.

query5 = """
SELECT
(
    feb_2026 - feb_2025
) / NULLIF(feb_2025,0) * 100 AS percentage_change
FROM
(
    SELECT
        (SELECT AVG(price_inr)
         FROM Crypto_prices
         WHERE coin_id = 'bitcoin'
         AND date BETWEEN '2025-02-01' AND '2025-02-28') AS feb_2025,

        (SELECT AVG(price_inr)
         FROM Crypto_prices
         WHERE coin_id = 'bitcoin'
         AND date BETWEEN '2026-02-01' AND '2026-02-28') AS feb_2026
) AS price_comparison;
"""

cursor.execute(query5)
print(cursor.fetchone())

(Decimal('-19.30096170986041'),)


In [59]:
#Find the highest oil price in the last 5 years.

query1 = """
SELECT MAX(price) AS highest_price
FROM oil_prices
WHERE date >= CURDATE() - INTERVAL 5 YEAR;
"""

cursor.execute(query1)
print(cursor.fetchone())


(Decimal('123.640000'),)


In [60]:
#Get the average oil price per year.

query2 = """
SELECT YEAR(date) AS year, AVG(price) AS avg_price
FROM oil_prices
GROUP BY YEAR(date)
ORDER BY year;
"""

cursor.execute(query2)
print(cursor.fetchall())


((2020, Decimal('39.1604365079')), (2021, Decimal('68.1350996016')), (2022, Decimal('94.9028685259')), (2023, Decimal('77.5765322581')), (2024, Decimal('76.6322400000')), (2025, Decimal('65.3881048387')))


In [ ]:
#Show oil prices during COVID crash (March–April 2020).

query3 = """
SELECT date, price
FROM oil_prices
WHERE date BETWEEN '2020-03-01' AND '2020-04-30'
ORDER BY date;
"""

cursor.execute(query3)
print(cursor.fetchall())


In [62]:
#Find the lowest price of oil in the last 10 years.

query4 = """
SELECT MIN(price) AS lowest_price
FROM oil_prices
WHERE date >= CURDATE() - INTERVAL 10 YEAR;
"""

cursor.execute(query4)
print(cursor.fetchone())


(Decimal('-36.980000'),)


In [63]:
#Calculate the volatility of oil prices (max-min difference per year).

query5 = """
SELECT 
    YEAR(date) AS year,
    MAX(price) - MIN(price) AS yearly_volatility
FROM oil_prices
GROUP BY YEAR(date)
ORDER BY year;
"""

cursor.execute(query5)
print(cursor.fetchall())


((2020, Decimal('100.250000')), (2021, Decimal('38.170000')), (2022, Decimal('52.590000')), (2023, Decimal('27.060000')), (2024, Decimal('20.960000')), (2025, Decimal('25.290000')))


In [ ]:
#Get all stock prices for a given ticker


tickers = ("^GSPC", "^IXIC", "^NSEI")

placeholders = ", ".join(["%s"] * len(tickers))

query1 = f"""
SELECT *
FROM Stock_prices
WHERE ticker IN ({placeholders})
ORDER BY date;
"""

cursor.execute(query1, tickers)
print(cursor.fetchall())


In [67]:
#Find the highest closing price for NASDAQ (^IXIC)

query2 = """
SELECT MAX(close) AS highest_close
FROM Stock_prices
WHERE ticker = '^IXIC';
"""

cursor.execute(query2)
print(cursor.fetchone())


(Decimal('23958.470703'),)


In [68]:
#List top 5 days with highest price difference (high - low) for S&P 500 (^GSPC)


query3 = """
SELECT date, high, low, (high - low) AS price_difference
FROM Stock_prices
WHERE ticker = '^GSPC'
ORDER BY price_difference DESC
LIMIT 5;
"""

cursor.execute(query3)
print(cursor.fetchall())


((datetime.date(2025, 4, 9), Decimal('5481.339844'), Decimal('4948.430176'), Decimal('532.909668')), (datetime.date(2025, 4, 9), Decimal('5481.339844'), Decimal('4948.430176'), Decimal('532.909668')), (datetime.date(2025, 4, 7), Decimal('5246.569824'), Decimal('4835.040039'), Decimal('411.529785')), (datetime.date(2025, 4, 7), Decimal('5246.569824'), Decimal('4835.040039'), Decimal('411.529785')), (datetime.date(2025, 4, 8), Decimal('5267.470215'), Decimal('4910.419922'), Decimal('357.050293')))


In [ ]:
#Get monthly average closing price for each ticker

query4 = """
SELECT 
    ticker,
    YEAR(date) AS year,
    MONTH(date) AS month,
    AVG(close) AS avg_monthly_close
FROM Stock_prices
GROUP BY ticker, YEAR(date), MONTH(date)
ORDER BY ticker, year, month;
"""

cursor.execute(query4)
print(cursor.fetchall())


In [72]:
#Get average trading volume of NSEI in 2024

query5 = """
SELECT AVG(volume) AS avg_volume_2024
FROM Stock_prices
WHERE ticker = '^NSEI'
AND YEAR(date) = 2024;
"""

cursor.execute(query5)
print(cursor.fetchone())


(Decimal('315994.9580'),)


In [75]:
#Compare Bitcoin vs Oil average price in 2025

query1 = """
SELECT 
    (SELECT AVG(price_inr)
     FROM Crypto_prices
     WHERE coin_id = 'bitcoin'
     AND YEAR(date) = 2025) AS btc_avg,

    (SELECT AVG(price)
     FROM oil_prices
     WHERE YEAR(date) = 2025) AS oil_avg;
"""

cursor.execute(query1)
print(cursor.fetchone())


(Decimal('8889394.5321375092'), Decimal('65.3881048387'))


In [78]:
#Check if Bitcoin moves with S&P 500 (Correlation Idea)

query2 = """
SELECT cp.date,
       cp.price_inr AS bitcoin_price,
       sp.close AS sp500_close
FROM Crypto_prices cp
JOIN Stock_prices sp
ON cp.date = sp.date
WHERE cp.coin_id = 'bitcoin'
AND sp.ticker = '^GSPC';
"""

cursor.execute(query2)
data = cursor.fetchall()

df = pd.DataFrame(data, columns=["date","btc","sp500"])
print(df["btc"].corr(df["sp500"]))


0.4778197517154566


In [ ]:
#Compare Ethereum and NASDAQ daily prices for 2025

query3 = """
SELECT cp.date,
       cp.price_inr AS ethereum_price,
       sp.close AS nasdaq_close
FROM Crypto_prices cp
JOIN Stock_prices sp
ON cp.date = sp.date
WHERE cp.coin_id = 'ethereum'
AND sp.ticker = '^IXIC'
AND YEAR(cp.date) = 2025;
"""

cursor.execute(query3)
print(cursor.fetchall())


In [81]:
#Find days when oil price spiked & compare with Bitcoin change

query4 = """
SELECT o.date,
       o.price AS oil_price,
       cp.price_inr AS bitcoin_price
FROM oil_prices o
JOIN Crypto_prices cp
ON o.date = cp.date
WHERE cp.coin_id = 'bitcoin'
AND o.price >
    (SELECT price * 1.05
     FROM oil_prices prev
     WHERE prev.date = DATE_SUB(o.date, INTERVAL 1 DAY));
"""

cursor.execute(query4)
print(cursor.fetchall())


((datetime.date(2025, 6, 13), Decimal('73.840000'), Decimal('9081227.025134')), (datetime.date(2025, 10, 23), Decimal('62.440000'), Decimal('9442499.946296')))


In [ ]:
#Compare top 3 coins daily trend vs Nifty (^NSEI)

query5 = """
SELECT cp.coin_id,
       cp.date,
       cp.price_inr,
       sp.close AS nifty_close
FROM Crypto_prices cp

JOIN (
    SELECT id
    FROM Cryptocurrencies
    ORDER BY market_cap DESC
    LIMIT 3
) top_coins
ON cp.coin_id = top_coins.id

JOIN Stock_prices sp
ON cp.date = sp.date

WHERE sp.ticker = '^NSEI';
"""

cursor.execute(query5)
print(cursor.fetchall())

In [ ]:
#Compare S&P 500 vs Crude Oil on same dates

query6 = """
SELECT sp.date,
       sp.close AS sp500_close,
       o.price AS oil_price
FROM Stock_prices sp
JOIN oil_prices o
ON sp.date = o.date
WHERE sp.ticker = '^GSPC';
"""

cursor.execute(query6)
print(cursor.fetchall())


In [85]:
# Correlate Bitcoin vs Oil (Daily Match)

query7 = """
SELECT cp.date,
       cp.price_inr AS btc_price,
       o.price AS oil_price
FROM Crypto_prices cp
JOIN oil_prices o
ON cp.date = o.date
WHERE cp.coin_id = 'bitcoin';
"""

cursor.execute(query7)
data = cursor.fetchall()

df = pd.DataFrame(data, columns=["date","btc","oil"])
print(df["btc"].corr(df["oil"]))

0.032287036819918


In [ ]:
#Compare NASDAQ vs Ethereum price trend

query8 = """
SELECT cp.date,
       cp.price_inr AS ethereum_price,
       sp.close AS nasdaq_close
FROM Crypto_prices cp
JOIN Stock_prices sp
ON cp.date = sp.date
WHERE cp.coin_id = 'ethereum'
AND sp.ticker = '^IXIC';
"""

cursor.execute(query8)
print(cursor.fetchall())



In [ ]:
#Join top 3 crypto coins with stock indices for 2025


query9 = """
SELECT cp.coin_id,
       cp.date,
       cp.price_inr,
       sp.ticker,
       sp.close
FROM Crypto_prices cp

JOIN (
    SELECT id
    FROM Cryptocurrencies
    ORDER BY market_cap DESC
    LIMIT 3
) top_coins
ON cp.coin_id = top_coins.id

JOIN Stock_prices sp
ON cp.date = sp.date

WHERE YEAR(cp.date) = 2025
ORDER BY cp.coin_id, cp.date;
"""
cursor.execute(query9)
print(cursor.fetchall())

In [ ]:
# Multi-join: stock prices, oil prices, and Bitcoin prices for daily comparison


query10 = """
SELECT cp.date,
       cp.price_inr AS bitcoin_price,

       MAX(CASE WHEN sp.ticker='^GSPC' THEN sp.close END) AS sp500,
       MAX(CASE WHEN sp.ticker='^IXIC' THEN sp.close END) AS nasdaq,
       MAX(CASE WHEN sp.ticker='^NSEI' THEN sp.close END) AS nifty,

       MAX(o.Price) AS oil_price

FROM Crypto_prices cp

LEFT JOIN Stock_prices sp
ON cp.date = sp.date

LEFT JOIN oil_prices o
ON cp.date = o.Date

WHERE cp.coin_id = 'bitcoin'

GROUP BY cp.date, cp.price_inr
ORDER BY cp.date;
"""
cursor.execute(query10)
print(cursor.fetchall())
#for row in data:
   # print(row)
